#**Importing Libraries & G-Drive Set Up**

In [ ]:
# To upload our datasets from our working directory we need to mount our drive contents to the colab environment. 
# For the code to do so you can search “mount” in code snippets or use the code given below. 
# Our entire drive contents are now mounted on colab at the location “/gdrive”.
from google.colab import drive
drive.mount('/gdrive')
#Change current working directory to gdrive
%cd /gdrive


Drive already mounted at /gdrive; to attempt to forcibly remount, call drive.mount("/gdrive", force_remount=True).
/gdrive


In [ ]:
!pip install vecstack

  Created wheel for vecstack: filename=vecstack-0.4.0-py3-none-any.whl size=19878 sha256=a8003252c3f42bb87e52a9dd85de6fc9cc1395e5d2d6379a65420bb4d37e89a6
  Stored in directory: /root/.cache/pip/wheels/28/fe/0c/fe8e43660e3316d7ce204e59a79a72246c0ae9b6c5c79841c8
Successfully built vecstack


In [ ]:
from vecstack import stacking
import pandas as pd
import numpy as np
from sklearn.metrics import accuracy_score #works
from sklearn.model_selection import train_test_split
from sklearn.ensemble import RandomForestClassifier
from sklearn.tree import DecisionTreeClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import RandomizedSearchCV
from sklearn.model_selection import cross_val_score
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.ensemble import GradientBoostingClassifier
from sklearn.datasets.samples_generator import make_blobs
from sklearn.neighbors import KNeighborsClassifier
from matplotlib import pyplot as plt
from sklearn.calibration import CalibratedClassifierCV
from sklearn.neural_network import MLPClassifier
#from sklearn.ensemble import RandomForestClassifier
from imblearn.over_sampling import SMOTE 
from sklearn.svm import SVC
from sklearn import svm
from collections import Counter #for Smote, 

import warnings
warnings.filterwarnings("ignore")


/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.datasets.samples_generator module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be imported from sklearn.datasets. Anything that cannot be imported from sklearn.datasets is now part of the private API.
  warnings.warn(message, FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/externals/six.py:31: FutureWarning: The module is deprecated in version 0.21 and will be removed in version 0.23 since we've dropped support for Python 2.7. Please rely on the official version of six (https://pypi.org/project/six/).
  "(https://pypi.org/project/six/).", FutureWarning)
/usr/local/lib/python3.7/dist-packages/sklearn/utils/deprecation.py:144: FutureWarning: The sklearn.neighbors.base module is  deprecated in version 0.22 and will be removed in version 0.24. The corresponding classes / functions should instead be impo

#**Exploratory Data Analysis**

In [ ]:

trainfile = r'/gdrive/My Drive/CIS 508/Assignment 3/RevisedHomesiteTrain1.csv'
train_data = pd.read_csv(trainfile)

#train_data 


testfile = r'/gdrive/My Drive/CIS 508/Assignment 3/RevisedHomesiteTest1.csv'
test_data = pd.read_csv(testfile)

#test_data 


print(train_data.shape)    
print(test_data.shape)    

(65000, 596)
(173836, 596)


In [ ]:
test_data.head()

,CoverageField11A,CoverageField11B,CoverageField1A,CoverageField1B,CoverageField2A,CoverageField2B,CoverageField3A,CoverageField3B,CoverageField4A,CoverageField4B,CoverageField5A,CoverageField5B,CoverageField6A,CoverageField6B,Field10,Field11,Field7,Field8,Field9,GeographicField10A,GeographicField10B,GeographicField11A,GeographicField11B,GeographicField12A,GeographicField12B,GeographicField13A,GeographicField13B,GeographicField14A,GeographicField14B,GeographicField15A,GeographicField15B,GeographicField16A,GeographicField16B,GeographicField17A,GeographicField17B,GeographicField18A,GeographicField18B,GeographicField19A,GeographicField19B,GeographicField1A,...,PropertyField7_R,PropertyField7_S,PropertyField7_T,PropertyField14_A,PropertyField14_B,PropertyField14_C,PropertyField14_D,PropertyField28_A,PropertyField28_B,PropertyField28_C,PropertyField28_D,PropertyField30_N,PropertyField30_Y,PropertyField31_K,PropertyField31_M,PropertyField31_N,PropertyField31_O,PropertyField32_N,PropertyField32_Y,PropertyField33_E,PropertyField33_F,PropertyField33_G,PropertyField33_H,PropertyField34_N,PropertyField34_Y,PropertyField36_N,PropertyField36_Y,PropertyField37_,PropertyField37_N,PropertyField37_Y,PropertyField38_N,PropertyField38_Y,GeographicField63_,GeographicField63_N,GeographicField63_Y,GeographicField64_CA,GeographicField64_IL,GeographicField64_NJ,GeographicField64_TX,GeographicField64
0,13,22,4,4,4,4,3,3,3,4,13,22,13,23,1487.0,1.3045,16,0.9364,0.0006,-1,25,9,18,13,18,9,18,-1,19,10,13,8,17,2,10,-1,20,19,18,25,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,0,1,0,1,0,0,0,1,1,0,0,0,1,0,0,0,0,IL
1,4,5,8,14,8,14,7,12,8,13,13,22,13,23,564.0,1.1886,11,0.9919,0.0038,-1,25,4,15,8,13,5,14,-1,13,9,11,5,12,2,16,-1,22,23,24,17,...,0,0,0,1,0,0,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
2,3,3,11,18,11,18,10,16,10,18,13,22,13,23,564.0,1.0670,15,0.8945,0.0038,-1,25,4,10,10,16,5,10,-1,11,11,18,5,13,2,13,-1,15,21,20,3,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,0,1,0,0,1,1,0,0,1,0,1,0,0,1,0,0,0,0,0,NJ
3,5,9,14,22,15,22,13,20,22,25,13,22,13,23,1113.0,1.2665,21,0.8870,0.0004,-1,25,24,24,23,23,24,24,25,25,23,22,24,25,2,7,-1,13,11,14,9,...,0,0,0,0,0,1,0,0,1,0,0,1,0,0,1,0,0,1,0,0,0,1,0,1,0,1,0,0,1,0,1,0,0,1,0,0,0,0,0,TX
4,12,21,4,5,4,5,4,4,4,5,13,22,13,23,935.0,1.0200,25,0.9153,0.0007,-1,25,2,3,3,5,2,5,-1,7,6,6,2,6,15,23,-1,17,4,8,2,...,1,0,0,0,0,1,0,0,1,0,0,1,0,0,0,0,1,0,1,0,0,0,1,1,0,0,1,0,1,0,1,0,0,1,0,0,0,0,0,CA


In [ ]:
test_data.columns

Index(['CoverageField11A', 'CoverageField11B', 'CoverageField1A',
       'CoverageField1B', 'CoverageField2A', 'CoverageField2B',
       'CoverageField3A', 'CoverageField3B', 'CoverageField4A',
       'CoverageField4B',
       ...
       'PropertyField38_N', 'PropertyField38_Y', 'GeographicField63_ ',
       'GeographicField63_N', 'GeographicField63_Y', 'GeographicField64_CA',
       'GeographicField64_IL', 'GeographicField64_NJ', 'GeographicField64_TX',
       'GeographicField64'],
      dtype='object', length=596)

In [ ]:
test_data.isnull().sum()

CoverageField11A        0
CoverageField11B        0
CoverageField1A         0
CoverageField1B         0
CoverageField2A         0
                       ..
GeographicField64_CA    0
GeographicField64_IL    0
GeographicField64_NJ    0
GeographicField64_TX    0
GeographicField64       0
Length: 596, dtype: int64

In [ ]:
#Drop categorical features without encoding
categoricalFeatures = ['GeographicField64_CA',
'GeographicField64_IL', 'GeographicField64_NJ',
'GeographicField64_TX']
subData = test_data.drop(columns = categoricalFeatures)

#Do one-hot encoding
dV = pd.get_dummies(test_data['GeographicField64'],
prefix= 'GeographicField64')
dV

#Dataframe Conversion
test_data = pd.concat([subData, dV], axis=1)
test_data = test_data.drop(columns='GeographicField64')

X_train = train_data.iloc[:, :-1].copy()
X_test = test_data.copy()
Y_train = train_data.iloc[:, -1].copy()

#**Models**

##Decision Tree Classifier

In [ ]:
#Create Decision Tree
clf = DecisionTreeClassifier()
clf.fit(X_train, Y_train)

#Check accuracy on training set
clf_pred_train = clf.predict(X_train)
print('TRAIN ACCURACY: ', accuracy_score(Y_train, clf_pred_train),'\n')
#Check accuracy on test set
clf_pred = clf.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(Y_train, clf_pred_train),'\n')
print('Classification Report: \n', classification_report(Y_train, clf_pred_train))

TRAIN ACCURACY:  1.0 

Confusion Matrix: 
 [[52738     0]
 [    0 26369]] 

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     26369

    accuracy                           1.00     79107
   macro avg       1.00      1.00      1.00     79107
weighted avg       1.00      1.00      1.00     79107



###Hyperparameter Tuning for Decision Tree Classifier

In [ ]:
#Hyperparameter tuning done for decision tree classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
clf_random = RandomizedSearchCV(clf,parameters,n_iter=15)
clf_random.fit(X_train, Y_train)
grid_parm=clf_random.best_params_
print(grid_parm)

#Using the parameters obtained from HyperParameterTuning in the DecisionTreeClassifier 
clf = DecisionTreeClassifier(**grid_parm)
clf.fit(X_train,Y_train)
clf_predict = clf.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for Decision Tree:{0:6f}".format(clf.score(X_train,Y_train)))

#get cross-validation report
clf_cv_score = cross_val_score(clf, X_train, Y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(clf_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Decision Tree: ",clf_cv_score.mean())

{'min_samples_split': 90, 'max_depth': 1}
accuracy Score (train) after hypertuning for Decision Tree:0.743158
=== All AUC Scores ===
[0.74554418 0.74810391 0.74611301 0.75739477 0.77474403 0.81636329
 0.81788017 0.81190301 0.81775314 0.81813279]


=== Mean AUC Score ===
Mean AUC Score - Decision Tree:  0.7853932292523318


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
clf_Probability = pd.DataFrame(clf.predict_proba(X_test))
clf_Probability.head()

,0,1
0,0.937493,0.062507
1,0.937493,0.062507
2,0.937493,0.062507
3,0.937493,0.062507
4,0.937493,0.062507


##Random Forest Classifier

In [ ]:
#Create Random Forest
rfc = RandomForestClassifier()
rfc.fit(X_train, Y_train)

#Check accuracy on training set
rfc_pred_train = rfc.predict(X_train)
print('TRAIN ACCURACY: ', accuracy_score(Y_train, rfc_pred_train),'\n')
#Check accuracy on test set
rfc_pred = rfc.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(Y_train, rfc_pred_train),'\n')
print('Classification Report: \n', classification_report(Y_train, rfc_pred_train))

TRAIN ACCURACY:  1.0 

Confusion Matrix: 
 [[52738     0]
 [    0 26369]] 

Classification Report: 
               precision    recall  f1-score   support

           0       1.00      1.00      1.00     52738
           1       1.00      1.00      1.00     26369

    accuracy                           1.00     79107
   macro avg       1.00      1.00      1.00     79107
weighted avg       1.00      1.00      1.00     79107



###Hyperparameter Tuning for Random Forest Classifier

In [ ]:
#Hyperparameter tuning done for random forest classifier
parameters={'min_samples_split' : range(10,100,10),'max_depth': range(1,20,2)}
rfc_random = RandomizedSearchCV(rfc,parameters,n_iter=15)
rfc_random.fit(X_train, Y_train)
grid_parm_rfc=rfc_random.best_params_
print(grid_parm_rfc)

#Using the parameters obtained from HyperParameterTuning in the RandomForestClassifier 
rfc = RandomForestClassifier(**grid_parm_rfc)
rfc.fit(X_train,Y_train)
rfc_predict = rfc.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for Random Forest:{0:6f}".format(rfc.score(X_train,Y_train)))

#get cross-validation report
rfc_cv_score = cross_val_score(rfc, X_train, Y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(rfc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Random Forest: ",rfc_cv_score.mean())

{'min_samples_split': 50, 'max_depth': 9}
accuracy Score (train) after hypertuning for Random Forest:0.906860
=== All AUC Scores ===
[0.90972824 0.92592542 0.92782245 0.9275544  0.94476111 0.99822765
 0.99836664 0.99851413 0.99832376 0.99731267]


=== Mean AUC Score ===
Mean AUC Score - Random Forest:  0.9626536462224508


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
rfc_Probability = pd.DataFrame(rfc.predict_proba(X_test))
rfc_Probability.head()

,0,1
0,0.927178,0.072822
1,0.799776,0.200224
2,0.840800,0.159200
3,0.963565,0.036435
4,0.802541,0.197459


##Support Vector Machines Classifier

In [ ]:
svc = svm.LinearSVC()
svc.fit(X_train, Y_train)
svc_pred_train=svc.predict(X_train)
print('TRAIN ACCURACY: ', accuracy_score(Y_train, svc_pred_train),'\n')
#Check accuracy on test set
svc_pred = svc.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(Y_train, svc_pred_train),'\n')
print('Classification Report: \n', classification_report(Y_train, svc_pred_train))

TRAIN ACCURACY:  0.6681456761095731 

Confusion Matrix: 
 [[52602   136]
 [26116   253]] 

Classification Report: 
               precision    recall  f1-score   support

           0       0.67      1.00      0.80     52738
           1       0.65      0.01      0.02     26369

    accuracy                           0.67     79107
   macro avg       0.66      0.50      0.41     79107
weighted avg       0.66      0.67      0.54     79107



###Hyperparameter Tuning for Support Vector Machines

In [ ]:
#Hyperparameter tuning done for support vector machines classifier
parameters={'C' : [0.8, 5, 20]}
svc_random = RandomizedSearchCV(svc,parameters,n_iter=15)
svc_random.fit(X_train, Y_train)
grid_parm_svc=svc_random.best_params_
print(grid_parm_svc)

#Using the parameters obtained from HyperParameterTuning in the SupportVectorMachines 
svc = svm.LinearSVC(**grid_parm_svc)
svc.fit(X_train,Y_train)
svc_predict = svc.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for SVM:{0:6f}".format(svc.score(X_train,Y_train)))

#get cross-validation report
svc_cv_score = cross_val_score(svc, X_train, Y_train, cv=10, scoring="roc_auc")
print("=== All AUC Scores ===")
print(svc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - SVM: ",svc_cv_score.mean())

{'C': 0.8}


In [ ]:
svcC=CalibratedClassifierCV(base_estimator=svc, cv=4)
svcC.fit(X_train, Y_train)

CalibratedClassifierCV(base_estimator=LinearSVC(C=0.8, class_weight=None,
                                                dual=True, fit_intercept=True,
                                                intercept_scaling=1,
                                                loss='squared_hinge',
                                                max_iter=1000,
                                                multi_class='ovr', penalty='l2',
                                                random_state=None, tol=0.0001,
                                                verbose=0),
                       cv=4, method='sigmoid')

In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
svc_Probability = pd.DataFrame(svcC.predict_proba(X_test))
svc_Probability.head()

,0,1
0,0.730740,0.269260
1,0.666535,0.333465
2,0.641424,0.358576
3,0.892686,0.107314
4,0.746676,0.253324


##K-Nearest Neighbors Classifier

In [ ]:
knc = KNeighborsClassifier()
knc.fit(X_train, Y_train)
knc_pred_train=knc.predict(X_train)
print('TRAIN ACCURACY: ', accuracy_score(Y_train, knc_pred_train),'\n')
#Check accuracy on test set
knc_pred = knc.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(Y_train, knc_pred_train),'\n')
print('Classification Report: \n', classification_report(Y_train, knc_pred_train))

TRAIN ACCURACY:  0.8244782383354191 

Confusion Matrix: 
 [[45142  7596]
 [ 6289 20080]] 

Classification Report: 
               precision    recall  f1-score   support

           0       0.88      0.86      0.87     52738
           1       0.73      0.76      0.74     26369

    accuracy                           0.82     79107
   macro avg       0.80      0.81      0.80     79107
weighted avg       0.83      0.82      0.83     79107



###Hyperparameter Tuning for K_Nearest Neighbor Classifier

In [ ]:
#Hyperparameter tuning done for K_Nearest Neighbor classifier
parameters={'n_neighbors' : range(10, 30, 10), 'weights': ['uniform','distance'], 'p' : (2, 4)}
knc_random = RandomizedSearchCV(knc,parameters,n_iter=5)
knc_random.fit(X_train, Y_train)
grid_parm_knc=knc_random.best_params_
print(grid_parm_knc)

#Using the parameters obtained from HyperParameterTuning in the K_NearestNeighborClassifier 
knc = KNeighborsClassifier(**grid_parm_knc)
knc.fit(X_train,Y_train)
knc_predict = knc.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for KNN:{0:6f}".format(knc.score(X_train,Y_train)))

#get cross-validation report
knc_cv_score = cross_val_score(knc, X_train, Y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(knc_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - KNN: ",knc_cv_score.mean())

{'weights': 'distance', 'p': 2, 'n_neighbors': 10}
accuracy Score (train) after hypertuning for KNN:1.000000
=== All AUC Scores ===
[0.24584204 0.02108384 0.09581253 0.13537593 0.11861541]


=== Mean AUC Score ===
Mean AUC Score - KNN:  0.12334594980387434


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
knc_Probability = pd.DataFrame(knc.predict_proba(X_test))
knc_Probability.head()

,0,1
0,0.653634,0.346366
1,0.633317,0.366683
2,0.376068,0.623932
3,0.774008,0.225992
4,0.639203,0.360797


##MultiLayer Perceptron Classifier

In [ ]:
mlp = MLPClassifier()
mlp.fit(X_train, Y_train)
mlp_pred_train=mlp.predict(X_train)
print('TRAIN ACCURACY: ', accuracy_score(Y_train, mlp_pred_train),'\n')
#Check accuracy on test set
mlp_pred = mlp.predict(X_test)
print('Confusion Matrix: \n', confusion_matrix(Y_train, mlp_pred_train),'\n')
print('Classification Report: \n', classification_report(Y_train, mlp_pred_train))

TRAIN ACCURACY:  0.7548004601362711 

Confusion Matrix: 
 [[34818 17920]
 [ 1477 24892]] 

Classification Report: 
               precision    recall  f1-score   support

           0       0.96      0.66      0.78     52738
           1       0.58      0.94      0.72     26369

    accuracy                           0.75     79107
   macro avg       0.77      0.80      0.75     79107
weighted avg       0.83      0.75      0.76     79107



###Hyperparameter Tuning for MultiLayer Perceptron Classifier

In [ ]:
#Hyperparameter tuning done for MultiLayer Perceptron classifier
parameters = {'hidden_layer_sizes':[(10,5,3), (20,7,3)], 'activation':['tanh', 'relu'], 
              'learning_rate':['constant', 'adaptive'], 'max_iter' :[100, 150]}
mlp_random = RandomizedSearchCV(mlp,parameters,n_iter=8)
mlp_random.fit(X_train, Y_train)
grid_parm_mlp=mlp_random.best_params_
print(grid_parm_mlp)

#Using the parameters obtained from HyperParameterTuning in the MLPClassifier 
mlp = MLPClassifier(**grid_parm_mlp)
mlp.fit(X_train,Y_train)
mlp_predict = mlp.predict(X_test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for MLP:{0:6f}".format(mlp.score(X_train,Y_train)))

#get cross-validation report
mlp_cv_score = cross_val_score(mlp, X_train, Y_train, cv=5, scoring="roc_auc")
print("=== All AUC Scores ===")
print(mlp_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - MLP: ",mlp_cv_score.mean())

{'max_iter': 100, 'learning_rate': 'constant', 'hidden_layer_sizes': (20, 7, 3), 'activation': 'relu'}
accuracy Score (train) after hypertuning for MLP:0.666667
=== All AUC Scores ===
[0.49361614 0.55280899 0.50075858 0.5        0.5       ]


=== Mean AUC Score ===
Mean AUC Score - MLP:  0.5094367421520516


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
mlp_Probability = pd.DataFrame(mlp.predict_proba(X_test))
mlp_Probability.head()

,0,1
0,0.665049,0.334951
1,0.665049,0.334951
2,0.665049,0.334951
3,0.665049,0.334951
4,0.665049,0.334951


#**SMOTE**

In [ ]:
print("___________________________________________________________________\nSMOTE\n")
print('Original dataset shape %s' % Counter(Y_train))
sm = SMOTE(sampling_strategy='float', ratio=0.5)
X_train, Y_train = sm.fit_resample(X_train, Y_train)
print('Resampled dataset shape %s' % Counter(Y_train))

___________________________________________________________________
SMOTE

Original dataset shape Counter({0: 52738, 1: 12262})
Resampled dataset shape Counter({0: 52738, 1: 26369})


# **ENSEMBLE METHODS STACKING**

In [ ]:
print("___________________________________________________________________________________________\nEnsemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier\n")

#models = [ svm.LinearSVC(), RandomForestClassifier(), DecisionTreeClassifier(), KNeighborsClassifier(), MLPClassifier() ]
models = [ DecisionTreeClassifier(min_samples_split= 70, max_depth= 9)  , 
          RandomForestClassifier(min_samples_split= 90, max_depth= 9) ,
          MLPClassifier(max_iter= 100, learning_rate= 'constant', 
                        hidden_layer_sizes= (20, 7, 3), activation= 'tanh') ,
          KNeighborsClassifier(n_neighbors= 3) ,
           svm.LinearSVC(max_iter= 700) ]
      
S_Train, S_Test = stacking(models,                   
                           X_train, Y_train, X_test,   
                           regression=False, 
     
                           mode='oof_pred_bag', 
       
                           needs_proba=False,
         
                           save_dir=None, 
            
                           metric=accuracy_score, 
    
                           n_folds=4, 
                 
                           stratified=True,
            
                           shuffle=True,  
            
                           random_state=0,    
         
                           verbose=2)

___________________________________________________________________________________________
Ensemble Methods Predictions using GradientBoosting, RandomForest and Decision Tree Classifier

task:         [classification]
n_classes:    [2]
metric:       [accuracy_score]
mode:         [oof_pred_bag]
n_models:     [5]

model  0:     [DecisionTreeClassifier]
    fold  0:  [0.92106993]
    fold  1:  [0.92015978]
    fold  2:  [0.92147444]
    fold  3:  [0.91980178]
    ----
    MEAN:     [0.92062648] + [0.00067335]
    FULL:     [0.92062649]

model  1:     [RandomForestClassifier]
    fold  0:  [0.90964251]
    fold  1:  [0.90423219]
    fold  2:  [0.90195682]
    fold  3:  [0.90508697]
    ----
    MEAN:     [0.90522962] + [0.00279285]
    FULL:     [0.90522963]

model  2:     [MLPClassifier]
    fold  0:  [0.66668352]
    fold  1:  [0.66668352]
    fold  2:  [0.66663296]
    fold  3:  [0.66666667]
    ----
    MEAN:     [0.66666667] + [0.00002064]
    FULL:     [0.66666667]

model  3:     [

###**Hyperparameter Tuning for Ensemble Methods Stacking**

In [ ]:
 
ems = GradientBoostingClassifier()
ems = ems.fit(S_Train, Y_train)
y_pred = ems.predict(S_Test)

In [ ]:
#Hyperparameter tuning done for stacked model
parameters = {'n_estimators': [100, 200, 350], 'loss': ['deviance', 'exponential'], 
              'criterion': ['friedman_mse', 'squared_error']}
ems_random = RandomizedSearchCV(ems,parameters,n_iter=5)
ems_random.fit(S_Train, Y_train)
grid_parm_ems=ems_random.best_params_
print(grid_parm_ems)

#Using the parameters obtained from HyperParameterTuning in the stacked model 
ems = GradientBoostingClassifier(**grid_parm_ems)
ems.fit(S_Train,Y_train)
ems_predict = ems.predict(S_Test)

#Obtain accuracy ,confusion matrix,classification report and AUC values for the result above.
print("accuracy Score (train) after hypertuning for Stacked Model:{0:6f}".format(ems.score(S_Train,Y_train)))

#get cross-validation report
ems_cv_score = cross_val_score(ems, S_Train, Y_train, cv=7, scoring="roc_auc")
print("=== All AUC Scores ===")
print(ems_cv_score)
print('\n')
print("=== Mean AUC Score ===")
print("Mean AUC Score - Stacked Model: ",ems_cv_score.mean())

{'n_estimators': 350, 'loss': 'exponential', 'criterion': 'friedman_mse'}
accuracy Score (train) after hypertuning for Stacked Model:0.926644
=== All AUC Scores ===
[0.90202689 0.88766902 0.88175228 0.96378    0.9951053  0.99388178
 0.9945264 ]


=== Mean AUC Score ===
Mean AUC Score - Stacked Model:  0.9455345256584576


In [ ]:
#Get Prediction Probability for the predicted class as a dataframe
stack_Probability = pd.DataFrame(ems.predict_proba(S_Test))

stack_Probability.head()

,0,1
0,0.948673,0.051327
1,0.948673,0.051327
2,0.948673,0.051327
3,0.966333,0.033667
4,0.948673,0.051327


# **Save Reults File**

In [ ]:
quotefile = r'/gdrive/My Drive/CIS 508/Assignment 3/QuoteNumber.csv'
qN = pd.read_csv(quotefile)

In [ ]:
results = qN['QuoteNumber']
results_df = pd.DataFrame(results, columns=['QuoteNumber'])

In [ ]:
#Stacked
results_df['QuoteConversion_Flag'] = stack_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/stacked_result.csv", index = None)

In [ ]:
#Decision Tree
results_df['QuoteConversion_Flag'] = clf_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/DT_result.csv", index = None)

In [ ]:
#Random Forest
results_df['QuoteConversion_Flag'] = rfc_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/RF_result.csv", index = None)

In [ ]:
#SVM
results_df['QuoteConversion_Flag'] = svc_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/SVM_result.csv", index = None)

In [ ]:
#MLP
results_df['QuoteConversion_Flag'] = mlp_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/MLP_result.csv", index = None)

In [ ]:
#KNC
results_df['QuoteConversion_Flag'] = knc_Probability.iloc[:, -1]
results_df.to_csv("/gdrive/My Drive/CIS 508/Assignment 3/KNC_result.csv", index = None)